In [1]:
import fastai
from fastai.vision import *
from experiments import *
from losses import *
from dataset import *
from metrics import *
from config import *
from fastai.callbacks import CSVLogger

%load_ext autoreload
%autoreload 2

torch.cuda.set_device(0)

In [2]:
random_seed(42)
allData = (SegItemListCustom.from_folder(MANGA109_PATH)
       .filter_by_func(lambda x: Path(get_segmentation(x)).exists() and Path(x).parent.name in (trainFolders)))

In [3]:
props = {'bs': 4, 'val_bs': 2, 'num_workers': 0}
paths = ['model/resnet34', 'loss/dice']

In [4]:
def isNotCsvLogger(c):
    if hasattr(c, '__name__') and c.__name__ == 'CSVLogger':
        return False
    if hasattr(c, 'func'):
        return isNotCsvLogger(c.func)
    return True

In [5]:
for path in paths:
    for index, dataset in enumerate(getDatasets(allData)):
        MODEL_PATH = Path(EXPERIMENTS_PATH) / path / str(index)
        if not (MODEL_PATH / 'final refined model.pkl').exists():
            random_seed(42)
            data = dataset.databunch(**props).normalize(imagenet_stats)
            random_seed(42)
            learn = load_learner(MODEL_PATH, 'final model.pkl')
            random_seed(42)
            learn.callback_fns = list(filter(isNotCsvLogger, learn.callback_fns)) + [partial(CSVLogger, filename = 'refined history')]
            learn.data = data
            learn.unfreeze()
            random_seed(42)
            learn.fit_one_cycle(5, 1e-4)
            learn.save('refined model')
            learn.export(file='final refined model.pkl')
    for index, dataset in enumerate(getDatasets(allData, crop=False, cutInHalf=False)):
        MODEL_PATH = Path(EXPERIMENTS_PATH) / path / str(index)            
        if not (MODEL_PATH / 'final refined model 2.pkl').exists():   
            random_seed(42)
            data = dataset.databunch(bs=1, val_bs = 1, num_workers=0).normalize(imagenet_stats)
            learn = load_learner(MODEL_PATH, 'final refined model.pkl')
            learn.callback_fns = list(filter(isNotCsvLogger, learn.callback_fns)) + [partial(CSVLogger, filename = 'refined history 2')]
            learn.data = data
            learn.unfreeze()
            random_seed(42)
            learn.fit_one_cycle(3, 1e-4)
            learn.save('refined model 2')
            learn.export(file='final refined model 2.pkl')
            
        if not (MODEL_PATH / 'final refined predictions.csv').exists():
            learn = load_learner(MODEL_PATH, 'final refined model.pkl')
            random_seed(42)
            m = getDatasetMetrics(dataset, learn)
            m.save(MODEL_PATH / 'final refined predictions.csv')
            
        if not (MODEL_PATH / 'final refined predictions 2.csv').exists():
            learn = load_learner(MODEL_PATH, 'final refined model 2.pkl')
            random_seed(42)
            m = getDatasetMetrics(dataset, learn)    
            m.save(MODEL_PATH / 'final refined predictions 2.csv')